# dp100_14 パイプラインを使用して機械学習を調整する

パイプラインという用語は機械学習で広く使用されており、多くの場合意味はさまざま。  
たとえば、scikit-learnでは、パイプラインを定義して、データ前処理変換とトレーニングアルゴリズムを組み合わせることができる。  
また、Azure DevOpsでは、ビルドまたはリリースのパイプラインを定義して、ソフトウェアの配布に必要なビルドと厚生のタスクを実行できる。  
この子ジュールで焦点を当てるのは**AzureMLパイプラインで、これは実験として実行できるステップをカプセル化するため**のもの。  
ただし、念頭に置くべき点として、Azure DevOpsパイプラインのなkのタスクによってAzureMLパイプラインを開始させ、  
次いでその中にscikit-learnパイプラインを基にモデルをトレーニングするステップを含めても一向に差し支えがない。

## パイプラインの概要

パイプラインは、各タスクが1つの"ステップ"として実装される機械学習タスクのワークフロー。

ステップを順番にまたは並列で配置することで、機械学習操作を調整する高度なフローロジックを作成できる。  
各ステップは特定のコンピューティング先で実行できるため、必要に応じてさまざまな種類のプロセスを組み合わせて、全体的な目標を達成することができる。

パイプラインは1つのプロセスとして実行できる。そのためには、パイプラインを実験として実行する。  
パイプライン実行の各ステップは、実験実行全体の一部として、それぞれに割り当てられたコンピューティング先で実行される。

パイプラインをRESTエンドポイントとして発行し、クライアントアプリケーションでパイプライン実行を開始できるようにすることができる。  
また、パイプラインのスケジュールを定義し、定期的に自動で実行するように設定することもできる。

### パイプラインのステップ

パイプラインは、タスクを実行する1つ以上の"ステップ"で構成される。  
AzureMLパイプラインではさまざまな種類のステップがサポートされており、それぞれに固有の目的と構成のオプションがある。

AzureMLパイプラインでの一般的なステップは次の通り。

- PythonScriptStep
    - 指定されたPythonスクリプトを実行する